In [56]:
from config import w_key
import requests
import json
import pandas as pd
import time

In [47]:
#Found a csv with station IDs, Latitude, and Longitude. 
#Changed the column names in excel so they'd work better with pandas.

path = "Resources/coops_active_current_stations.csv"
curr_csv = pd.read_csv(path)
#curr_csv = csv.rename(columns={"Station Name": "Station_Name", "Station ID": "Station_ID"})
curr_csv

,Station_ID,Station_Name,Latitude,Longitude,Project
0,ca0101,Cape Cod Canal West End,41.737499,-70.624901,Cape Cod PORTS
1,cb0102,Cape Henry LB 2CH,36.959400,-76.012800,Chesapeake Bay South PORTS
2,cb0201,York Spit LBB 22 (ATON),37.140000,-76.138300,Chesapeake Bay South PORTS
3,cb0301,Thimble Shoal LB 18,37.011080,-76.249030,Chesapeake Bay South PORTS
4,cb0402,Naval Station Norfolk LB 7,36.962950,-76.333880,Chesapeake Bay South PORTS
...,...,...,...,...,...
60,sn0701,Port Arthur,29.866700,-93.931110,Sabine Neches PORTS
61,t01010,Sunshine Skyway Bridge ADCP,27.620650,-82.655250,Tampa Bay PORTS
62,t02010,Old Port Tampa,27.862870,-82.553730,Tampa Bay PORTS
63,t03010,Port Manatee,27.663950,-82.598770,Tampa Bay PORTS


In [5]:
#TO TEST A DIFFERENT TYPE OF CURRENT API CALL.... Ignore this one

url = f"https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9414290/nearby.json?radius=3"
#url = f"https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/lat/51.883583/lng/-176.642481/nearby.json?radius=3"
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "count": 1,
    "self": "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9414290/nearby.json",
    "stations": [
        {
            "affiliations": "",
            "disclaimers": null,
            "distance": 0.0,
            "expand": null,
            "id": "9414290",
            "lat": 37.80630555555555,
            "lng": -122.4658888888889,
            "name": "San Francisco, CA",
            "notices": null,
            "portscode": "",
            "products": null,
            "self": "https://api.tidesandcurrents.noaa.gov/mdapi/prod/webapi/stations/9414290.json",
            "stationType": "Water Level,Meteorological,Physical",
            "tideType": ""
        }
    ],
    "units": null
}


In [6]:
#TO TEST STATION IDS
url = f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date=today&station=ca0101&product=currents_predictions&units=metric&time_zone=gmt&application=ports_screen&format=json"
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))
#print(response["current_predictions"]["cp"][0]["Velocity_Major"])

{
    "current_predictions": {
        "cp": [
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:00",
                "Velocity_Major": 164.1,
                "meanEbbDir": 217,
                "meanFloodDir": 41
            },
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:06",
                "Velocity_Major": 164.3,
                "meanEbbDir": 217,
                "meanFloodDir": 41
            },
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:12",
                "Velocity_Major": 164.4,
                "meanEbbDir": 217,
                "meanFloodDir": 41
            },
            {
                "Bin": "20",
                "Depth": "4",
                "Time": "2020-12-02 00:18",
                "Velocity_Major": 164.3,
                "meanEbbDir": 217,
                "meanFloodDir":

In [43]:
#Tide query
#Grabs tide data for the current day
date = "today"
product = "currents_predictions"


t_stations = curr_csv["Station_ID"].to_list()
t_locations = curr_csv["Station_Name"].to_list()
t_lats = curr_csv["Latitude"].to_list()
t_lons = curr_csv["Longitude"].to_list()

datum = "MTL"
unit = "metric"
time = "lst"
format_j = "json"
app = "app"

#Outputs
velo = []
ebb = []
flood = []
s_result = []
locate = []
LAT = []
LON = []


for (t_station, t_location, t_lat, t_lon) in zip(t_stations, t_locations, t_lats, t_lons):
    try:
        t_url = f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date={date}&station={t_station}&product={product}&units={unit}&time_zone={time}&application={app}&format={format_j}"
        #t_url = f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?date={date}&station={station}&product={product}&datum={datum}&units={unit}&time_zone={time}&application={app}&format={format_j}"
        t_resp = requests.get(t_url).json()
        velocity = t_resp["current_predictions"]["cp"][0]["Velocity_Major"] #In cm/second
        tide_ebb = t_resp["current_predictions"]["cp"][0]["meanEbbDir"] #degrees azimuth (360)
        tide_flood = t_resp["current_predictions"]["cp"][0]["meanFloodDir"] #degrees azimuth
        velo.append(velocity)
        ebb.append(tide_ebb)
        flood.append(tide_flood)
        s_result.append(t_station)
        locate.append(t_location)
        LAT.append(t_lat)
        LON.append(t_lon)
    except KeyError:
        print(t_station)

cc0201
g10010
gl0101
gl0201
lc0101
lm0101
lm0201
mb0302
mc0101
mi0101
mi0201
sn0101
sn0401
sn0601
t04010


In [4]:
velo

[-106.1,
 12.2,
 32.3,
 6.7,
 -18.4,
 -9.9,
 -9.5,
 17.2,
 -19.8,
 -40.3,
 -33.4,
 29.4,
 82.5,
 31,
 -15.8,
 33.4,
 -26.5,
 11.9,
 -1.3,
 15.4,
 56.6,
 -9.6,
 21.6,
 33.5,
 55.6,
 84.4,
 -24.7,
 -78.5,
 -60.3,
 -45.2,
 13.1,
 -7.8,
 43,
 52.3,
 -48.3,
 -55.3,
 -12.1,
 -26.8,
 -3.9,
 1.5,
 78.9,
 31.6,
 69.4,
 27.5,
 3.3,
 17.1,
 40.7,
 61.6,
 51.5,
 58.5]

In [65]:
#Makes a dataframe for tide data.

tides = {"Current_Station_ID": s_result, "Station_Name": locate, "Latitude": LAT,
         "Longitude": LON, "Velocity(cm/s)": velo,
         "Ebb_Direction(Degrees)": ebb, "Flood_Direction(Degrees)": flood}
tide_df = pd.DataFrame(tides, columns=["Current_Station_ID", "Station_Name", "Latitude", "Longitude",
                                       "Velocity(cm/s)", "Ebb_Direction(Degrees)", "Flood_Direction(Degrees)"])

tide_df.to_csv('Resources/my_tides.csv', index = False)
tide_df

,Current_Station_ID,Station_Name,Latitude,Longitude,Velocity(cm/s),Ebb_Direction(Degrees),Flood_Direction(Degrees)
0,ca0101,Cape Cod Canal West End,41.737499,-70.624901,-106.1,217,41
1,cb0102,Cape Henry LB 2CH,36.959400,-76.012800,12.2,112,297
2,cb0201,York Spit LBB 22 (ATON),37.140000,-76.138300,32.3,168,347
3,cb0301,Thimble Shoal LB 18,37.011080,-76.249030,6.7,84,286
4,cb0402,Naval Station Norfolk LB 7,36.962950,-76.333880,-18.4,10,202
5,cb0601,Newport News Channel LB 14,36.956020,-76.414480,-9.9,100,285
6,cb0701,Dominion Terminal,36.962330,-76.424170,-9.5,118,309
7,cb0801,Rappahannock Shoal Channel LBB 60,37.674420,-76.154720,17.2,175,337
8,cb1001,Cove Point LNG Pier,38.402830,-76.384170,-19.8,118,318
9,cb1101,Chesapeake Channel LBB 92,38.982000,-76.385000,-40.3,226,23


In [67]:
#Open weather query. Made a for loop to query weather data for each latitude and longitude in above dataframe.
#To avoid getting locked out of the weather api, I made use of time sleep to iterate in one second intervals. 

units = "standard"
lats = tide_df["Latitude"].to_list()
lons = tide_df["Longitude"].to_list()

#lats = ["51.883583"]
#lons = ["-176.642481"]

#Outputs
temps = []
wind_speed = []
w_lat = []
w_lon = []



for (lat, lon) in zip(lats, lons):
    w_url= f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&units={units}&appid={w_key}"
    w_query= requests.get(w_url).json()
    temp_get = w_query['main']['temp']
    wind_get = w_query['wind']['speed']
    temps.append(temp_get)
    wind_speed.append(wind_get)
    w_lat.append(lat)
    w_lon.append(lon)
    time.sleep(1)

In [68]:
#This is the dataframe for weather which will match to the tide station df through coordinates. 

weather = {"Latitude": w_lat,"Longitude": w_lon, "Temp(F)": temps, "Wind_Speed(mph)": wind_speed}
weather_df = pd.DataFrame(weather, columns=["Latitude", "Longitude",
                                       "Temp(F)", "Wind_Speed(mph)"])
weather_df.to_csv('Resources/my_weather.csv', index = False)
weather_df

,Latitude,Longitude,Temp(F),Wind_Speed(mph)
0,41.737499,-70.624901,278.89,4.10
1,36.959400,-76.012800,280.78,2.60
2,37.140000,-76.138300,282.97,4.02
3,37.011080,-76.249030,281.99,2.10
4,36.962950,-76.333880,281.58,2.10
5,36.956020,-76.414480,281.64,2.10
6,36.962330,-76.424170,281.68,2.10
7,37.674420,-76.154720,279.74,0.45
8,38.402830,-76.384170,280.05,2.60
9,38.982000,-76.385000,281.03,2.60
